In [1]:
#pip install pymo

In [2]:
#Load packages
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import pickle
import timeit
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
import scipy.io as sio
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import warnings
warnings.filterwarnings("ignore")

In [3]:

#clase principal shecco
class SHECCO(BaseEstimator,TransformerMixin):
    def __init__(self, sigmax = 54, sigmay = 54, Ax = 1, Ay = 1,ws = 5, noverlap = None, k = 50, sup = 1):
        self.sigmax = sigmax
        self.sigmay = sigmay
        self.ws = ws
        self.noverlap = noverlap
        self.sup = sup
        self.k = k
        self.Ax = Ax
        self.Ay = Ay
        
##################################################################

    def plot_mocap(self, Ns, Nv, ind):
     mp_p = MocapParameterizer(param_type = 'position')
     #print("prueba!")
     #print(list_folder)
     print("NS")
     print(Ns)
     print("list")
     print(list_folder)
     print(Ns)
     data_all = [p.parse('D:/Escritorio/datos_mocap/'+Ns+'/'+list_folder[Nv])]
     pos_d= mp_p.fit_transform(data_all)
     #Normalization
     n=pos_d[0].values
     P = np.asarray(pos_d[0].values)
     print("Shape of P:", P.shape)
     #z = P.reshape(n.shape[0],38,3)
     #z = P.reshape(2999, 38, 3)
     z = P.reshape(2999, 73)
     
     Z = np.zeros(z.shape)
     for i in range(z.shape[0]):
       for j in range(z.shape[1]):
         Z[i,j,:] =z[i,j,:]- z[i,0,:]       
    
     print(f"Shape of ind: {ind.shape}")
     print(f"Shape of Z: {Z.shape}")
    
     #Distance among skeletons
     for k in range(ind.shape[0]):
       for l in range(Z.shape[1]):
         Z[ind[k],l,0]= Z[ind[k],l,0]+20*k
     Z = Z.reshape(n.shape[0],114)
     n.iloc[:]=Z[:]
     pos_d[0].values = n

     if ind.shape[0]<= 10:
       draw_stickfigure(pos_d[0], frame = ind, figsize=(20,4))
     else: 
       nf = int(ind.shape[0]/10)
       for h in range(0,nf):
         draw_stickfigure(pos_d[0], frame = ind[10*h:10*(h+1)], figsize=(20,4))
         plt.show()

########################################

    def geneout_(self,X):
      #print(X.shape)
      N,T = X.shape
      Y = np.zeros((N,T))
      for n in range(N):
        for t in range(T-1):
          Y[n,t]=X[n,t+1]
      return Y

    def ventaneo_(self,X):
       N,P = X.shape
       if self.noverlap == None: self.noverlap =0
       if self.ws == None: self.ws = P
       for j in range(N):
         pi = 0
         pf = self.ws
         Xi = X[j,pi:pf].reshape(1,-1) 
         Yi = X[j,pf].reshape(1,-1)
         pi = pi + (self.ws - self.noverlap)
         pf = pi +self.ws
         while pf  < P:
                Xi = np.r_[Xi,X[j,pi:pf].reshape(1,-1)]
                Yi = np.r_[Yi,X[j,pf].reshape(1,-1)]
                pi = pi + (self.ws - self.noverlap)
                pf = pi +self.ws
                #print(Xi,Yi)
         #print(pi,pf)
         if pf >= P:
            temp = X[j,pi:].reshape(1,-1)
            Xi = np.r_[Xi,np.c_[temp,np.zeros(self.ws-temp.shape[1]).reshape(1,-1)]]
            Yi = np.r_[Yi,np.zeros(1).reshape(-1,1)]       
         if j == 0: 
           #print(Xi.shape)
           X_ = np.zeros((N,Xi.shape[0],Xi.shape[1]))
           Y_ = np.zeros((N,Yi.shape[0],1))
         X_[j] = Xi  
         Y_[j] = Yi  
       return X_,Y_  

    def SC_(self,X,Y):
        Xcl = np.zeros((X.shape[0], self.k))
        Ycl = np.zeros((Y.shape[0], self.k))
        for n in range(X.shape[0]):
            Z = X[n]
            L = Y[n]
            Z = np.tile(Z,(Z.shape[0],1))
            D = pairwise_distances(Z,Z)
            gamma = 1/(np.median(D)**2)
            K = np.exp(-0.5*gamma*D**2)
            Clus = SpectralClustering(n_clusters= self.k,affinity='precomputed')
            y_pred = Clus.fit_predict(K)
            centers = np.zeros((self.k,Z.shape[1]))
            for c in range(self.k):
                centers[c] = Z[y_pred==c].mean(axis=0)
                
            Ds = pairwise_distances(centers,Z)
            ind = np.argmin(Ds,axis=1) 
            Xcl[n] = Z[ind]
            Ycl[n] = L[ind]
        return Xcl,Ycl

    def SC_Mocap(self,X,Y):
      Xcl = np.zeros((self.k, X.shape[1]))
      Ycl = np.zeros((self.k, Y.shape[1]))
      D = pairwise_distances(Y,Y)
      gamma = 1/(np.median(D)**2)
      K = np.exp(-0.5*gamma*D**2)
      Clus = SpectralClustering(n_clusters= self.k,affinity='precomputed')
      y_pred = Clus.fit_predict(K)
      centers = np.zeros((self.k,Y.shape[1]))
      for c in range(self.k):
        centers[c] = Y[y_pred==c].mean(axis=0)

      Ds = pairwise_distances(centers,Y)
      ind = np.argmin(Ds,axis=1)
      Xcl = X[ind]
      Ycl = Y[ind]
      return Xcl,Ycl,ind


    def Kmeans_Mocap(self,X):
      Xcl = np.zeros((self.k, X.shape[1]))
      #Ycl = np.zeros((self.k, Y.shape[1]))
      #Clus = KMeans(n_clusters= self.k, random_state=0)
      y_pred = KMeans(n_clusters= self.k, random_state=0).fit_predict(X)
      centers = np.zeros((self.k,X.shape[1]))
      for c in range(self.k):
        centers[c] = X[y_pred==c].mean(axis=0)

      Ds = pairwise_distances(centers,X)
      ind = np.argmin(Ds,axis=1)
      Xcl = X[ind]
      #Ycl = Y[ind]
      return Xcl,ind

    def kergM(self,X,Y,sigma_,A):
       return A*np.exp(-pairwise_distances(X,Y)**2/(2*sigma_**2))
    
    def shecco_(self,X,Y,Z,L,pxx,pyy,pzz,pll,pxz=None,pyl=None):  
      #se asume independencia
     
      if (pxz==None) & (pyl==None):
          PXX = pxx.reshape(-1,1)#.dot(pxx.reshape(1,-1))
          PYY = pyy.reshape(-1,1)#.dot(pyy.reshape(1,-1))  
          PZZ = pzz.reshape(-1,1)#.dot(pzz.reshape(1,-1))  
          PLL = pll.reshape(-1,1)#.dot(pll.reshape(1,-1))  
          PXY = pxx.reshape(-1,1)#.dot(pyy.reshape(1,-1))  
          PZL = pzz.reshape(-1,1)#.dot(pll.reshape(1,-1))  
      KXX  = self.kergM(X,X,self.sigmax,self.Ax)
      KZZ  = self.kergM(Z,Z,self.sigmax,self.Ax)
      KYY  = self.kergM(Y,Y,self.sigmay,self.Ay)
      KLL  = self.kergM(L,L,self.sigmay,self.Ay)
      KXZ  = self.kergM(X,Z,self.sigmax,self.Ax)
      KYL  = self.kergM(Y,L,self.sigmay,self.Ay)


      #trxx=KXX.shape[0]**2 #np.sqrt(np.trace((KXX.dot(KXX))))
  
      #trzz=KZZ.shape[0]**2#np.sqrt(np.trace((KZZ.dot(KZZ))))
    
      #tryy=KYY.shape[0]**2#np.sqrt(np.trace((KYY.dot(KYY))))
     
      #trll=KLL.shape[0]**2#np.sqrt(np.trace((KLL.dot(KLL))))
    
      #trxz=KXX.shape[0]*KZZ.shape[0]#np.sqrt(np.trace((KXZ.dot(KXZ.T))))
     
      #tryl=KYY.shape[0]*KLL.shape[0]#np.sqrt(np.trace((KYL.dot(KYL.T))))
      #KXX = KXX*trxx
      #KZZ = KZZ*trzz
      #KYY = KYY*tryy
      #KLL = KLL*trll
      #KXZ = KXZ*trxz
      #KYL = KYL*tryl
     
      #d = np.trace((PXX*KXX).dot(KYY*PYY)) -2*np.trace((PXZ*KXZ).dot((KYL*PYL).T)) + np.trace((PZZ*KZZ).dot(KLL*PLL))
      d = (PXY.T).dot(KXX*KYY).dot(PXY) -2*(PXY.T).dot(KXZ*KYL).dot(PZL) + (PZL.T).dot(KZZ*KLL).dot(PZL)

      if d<0:
        d = abs(d)
      return d

    #hecco sin salida (no supervised)
    def hecco_(self,X,Z,pxx,pzz,pxz=None,pyl=None):  
      #se asume independencia
     
      if (pxz==None) & (pyl==None):
          PXX = pxx.reshape(-1,1).dot(pxx.reshape(1,-1))
          PZZ = pzz.reshape(-1,1).dot(pzz.reshape(1,-1))   
          PXZ = pxx.reshape(-1,1).dot(pzz.reshape(1,-1)) 
      KXX  = self.kergM(X,X,self.sigmax)
      KZZ  = self.kergM(Z,Z,self.sigmax)
      KXZ  = self.kergM(X,Z,self.sigmax)
      
      d = np.dot(np.dot(np.ones(PXX.shape[0]).T,(PXX*KXX)),np.ones(PXX.shape[0]))-2*np.dot(np.dot(np.ones(PXX.shape[0]).T,(PXZ*KXZ)),np.ones(PZZ.shape[0]))+ np.dot(np.dot(np.ones(PZZ.shape[0]).T,(PXX*KXX)),np.ones(PZZ.shape[0]))
     
      if d<0:
        d = abs(d)
      return d   

    def SHECCO_M_(self,Xtrain_,Xtest_,Ytrain_,Ytest_,pzz,pll):
      Ntrain = Xtrain_.shape[0]
      Ntest = Xtest_.shape[0]
        
      Dte_tr = np.zeros((Ntest,Ntrain))
      for i in range(Ntrain):
        for j in range(Ntest):
          if self.sup == None:
              Dte_tr[j,i] = self.hecco_(Xtrain_[i],Xtest_[j],self.pxx,pzz)
          else:
            Dte_tr[j,i] = self.shecco_(Xtrain_[i],Ytrain_[i],Xtest_[j],Ytest_[j],
                       self.pxx,self.pyy,pzz,pll)
      return Dte_tr

    def fit(self,X,*_):
        #self.Xtrain_,self.Ytrain_ = self.ventaneo_(X)
        # If we want use Spectral Clustering:
        #if self.k != None:
           # self.Xtrain_, self.Ytrain_= self.SC_(self.Xtrain_,self.Ytrain_)
        self.Xtrain_= X
        Ya = np.zeros((X.shape[0],self.k,114))
        for i in range(X.shape[0]):
          Z = X[i]
          L = dist.geneout_(Z.T)
          Ya[i,:,:]= L.T
        Yp = Ya.reshape(X.shape[0]*X.shape[1],X.shape[2])
        pca = PCA(n_components=3)
        Ytr = pca.fit(Yp).transform(Yp)  
        Y = Ytr.reshape(X.shape[0], X.shape[1], Ytr.shape[1])
        self.Ytrain_= Y
        self.N = self.Xtrain_.shape[1]
        #estimar pdfs
        self.pxx = (1/self.N)*np.ones(self.N) #prob uniformes
        self.pyy = (1/self.N)*np.ones(self.N)
        return self

    def transform(self,X): #Xt_=[Xt,Yt]
        #Xtest_,Ytest_ = self.ventaneo_(X)
        # If we want use Spectral Clustering:
        #if self.k != None:
        Xtest_ = X
        Yb = np.zeros((X.shape[0],self.k,114))
        for i in range(X.shape[0]):
          Z = X[i]
          L = dist.geneout_(Z.T)
          Yb[i,:,:]= L.T
        Yp = Yb.reshape(X.shape[0]*X.shape[1],X.shape[2])
        pca = PCA(n_components=3)
        Yte = pca.fit(Yp).transform(Yp)  
        Y = Yte.reshape(X.shape[0], X.shape[1], Yte.shape[1])
        Ytest_ = Y
        M = Xtest_.shape[1]
        #estimar pdfs
        pzz = (1/M)*np.ones(M) #porbas uniformes
        pll = (1/M)*np.ones(M)
        return self.SHECCO_M_(self.Xtrain_,Xtest_,self.Ytrain_,Ytest_,pzz,pll)

    def fit_tranform(self,X):
        self.fit(X)
        return self.transform(X)

In [4]:
class MocapTrackWrapper:
    def __init__(self, df):
        self.df = df
        self.skeleton = self._generate_skeleton()

    def _generate_skeleton(self):
        # Aquí generamos un esqueleto simulado basado en las columnas del DataFrame
        # Suponiendo que los nombres de las columnas siguen un formato específico
        skeleton = {}
        joints = set(col.split('_')[0] for col in self.df.columns if '_Xposition' in col)
        for joint in joints:
            skeleton[joint] = {
                'Xposition': f'{joint}_Xposition',
                'Yposition': f'{joint}_Yposition'
            }
        return skeleton

    def __getitem__(self, item):
        return self.df[item]
    
    def __getattr__(self, name):
        if name == 'skeleton':
            return self._generate_skeleton()
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class SHECCO(BaseEstimator, TransformerMixin):
    def __init__(self, sigmax=54, sigmay=54, Ax=1, Ay=1, ws=5, noverlap=None, k=50, sup=1):
        self.sigmax = sigmax
        self.sigmay = sigmay
        self.ws = ws
        self.noverlap = noverlap
        self.sup = sup
        self.k = k
        self.Ax = Ax
        self.Ay = Ay

    def adjust_dataframe_columns(self, df):
        # Lista de articulaciones esperadas
        required_joints = ['Hips', 'Knees', 'Ankles']  # Agrega aquí todas las articulaciones necesarias
        for joint in required_joints:
            if f'{joint}_Xposition' not in df.columns:
                # Crear columnas faltantes con valores NaN
                df[f'{joint}_Xposition'] = np.nan
                df[f'{joint}_Yposition'] = np.nan
        return df

    def plot_mocap(self, Ns, Nv, ind):
        mp_p = MocapParameterizer(param_type='position')
        data_all = [p.parse(f'D:/Escritorio/datos_mocap/{Ns}/{list_folder[Nv]}')]
        pos_d = mp_p.fit_transform(data_all)
    
        # Convertir pos_d[0] a un DataFrame si no lo es
        if not isinstance(pos_d[0], pd.DataFrame):
            df = pd.DataFrame(pos_d[0].values)
        else:
            df = pos_d[0]
    
        # Ajustar el DataFrame para tener las columnas adecuadas
        df = self.adjust_dataframe_columns(df)
    
        # Envolver el DataFrame en el wrapper
        mocap_track = MocapTrackWrapper(df)
    
        # Normalización
        n = df.values
        P = np.asarray(df.values)
        print("Shape of P:", P.shape)
        print(Ns)
    
        # Ajustar el reshaping de P
        try:
            # En lugar de reshaping a (2999, 69), usar la forma original (2999, 73)
            z = P.reshape(2999, 73)
        except ValueError as e:
            print(f"Error reshaping P: {e}")
            return
    
        Z = np.zeros(z.shape)
        for i in range(z.shape[0]):
            for j in range(z.shape[1]):
                Z[i, j] = z[i, j] - z[i, 0]
    
        print(f"Shape of ind: {ind.shape}")
        print(f"Shape of Z: {Z.shape}")
    
        # Distancia entre esqueletos
        for k in range(ind.shape[0]):
            for l in range(Z.shape[1]):
                Z[ind[k], l] += 20 * k
    
        # Actualizar el DataFrame con el nuevo formato
        n = pd.DataFrame(Z, columns=df.columns)
        pos_d[0] = n
    
        # Llamar a draw_stickfigure con los datos ajustados
        if ind.shape[0] <= 10:
            draw_stickfigure(mocap_track, frame=ind, figsize=(20, 4))
        else:
            nf = int(ind.shape[0] / 10)
            for h in range(nf):
                draw_stickfigure(mocap_track, frame=ind[10 * h:10 * (h + 1)], figsize=(20, 4))
                plt.show()



In [6]:
import os
from pymo.parsers import BVHParser
from pymo.preprocessing import MocapParameterizer
import numpy as np

# Asegúrate de que list_folder está definido y contiene elementos
list_folder = os.listdir('D:/Escritorio/datos_mocap/')  # Asumiendo que list_folder debe contener los nombres de carpetas

p = BVHParser()
print(p)
mp = MocapParameterizer(param_type='position')
pos_cent = {}
directory_path = 'D:/Escritorio/datos_mocap/'
print(f"Contents of {directory_path}:")

try:
    contents = os.listdir(directory_path)
    for item in contents:
        print(item)
except PermissionError as e:
    print(f"Permission error for directory {directory_path}: {e}")
except FileNotFoundError as e:
    print(f"Directory not found: {directory_path}: {e}")

print(f"Checking files in list_folder:")
for i in range(len(list_folder)):
    dir_path = os.path.join(directory_path, list_folder[i])
    if os.path.isdir(dir_path):
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if os.path.isfile(file_path):
                try:
                    data_all = [p.parse(file_path)]
                    pos_cent[i] = mp.fit_transform(data_all)
                    print(f"Successfully processed {file_path}")
                except PermissionError as e:
                    print(f"Permission error for {file_path}: {e}")
                except FileNotFoundError as e:
                    print(f"File not found: {file_path}: {e}")
                except Exception as e:
                    print(f"An error occurred while processing {file_path}: {e}")
    else:
        print(f"{dir_path} is not a directory.")

# Normalization with respect to hips positions
for k in range(len(pos_cent)): 
    if k in pos_cent:
        n = pos_cent[k][0].values  # Recorriendo uno a uno los datos y solo traiga la posición 0
        p = np.asarray(n)
        X = p.reshape(n.shape[0], 69, 1)
        Z = np.zeros(X.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i, j, :] = X[i, j, :] - X[i, 0, :]
        Z = Z.reshape(n.shape[0], 69)
        n.iloc[:] = Z[:]
        pos_cent[k][0].values = n
        print(f"Normalized data for key {k}:")
        print(pos_cent[k][0].values)
    else:
        print(f"Key {k} not found in pos_cent")

# Imprime las claves de pos_cent para depuración
print(f"Keys in pos_cent: {list(pos_cent.keys())}")


Contents of D:/Escritorio/datos_mocap/:
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
desktop.ini
Checking files in list_folder:
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_Derecha.bvh
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_Remate.bvh
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_Reves.bvh
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_Servicio.bvh
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_VDerecha.bvh
Successfully processed D:/Escritorio/datos_mocap/01\adorozco_VReves.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_Derecha.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_Remate.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_Reves.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_Servicio.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_VDerecha.bvh
Successfully processed D:/Escritorio/datos_mocap/02\astrejos_VReves.bvh


In [7]:
directory_path

'D:/Escritorio/datos_mocap/'

In [8]:
p

array([[ -41.34528   ,   88.835663  ,  -54.858303  , ...,  -43.36520089,
         163.8526453 ,  -56.79047822],
       [ -41.340302  ,   88.831886  ,  -54.830879  , ...,  -43.69152108,
         163.87715688,  -56.59287911],
       [ -41.331707  ,   88.812065  ,  -54.802589  , ...,  -43.67851957,
         163.85443447,  -56.56796354],
       ...,
       [ -98.395172  ,   87.846031  ,  -39.082344  , ..., -133.58182228,
         133.90858458,    2.61317419],
       [ -98.395172  ,   87.846031  ,  -39.082344  , ..., -133.58178954,
         133.90869169,    2.61313556],
       [ -98.395172  ,   87.846031  ,  -39.082344  , ..., -133.58174339,
         133.90862967,    2.6132228 ]])

In [9]:
pos_cent

{0: [<pymo.data.MocapData at 0x11c2968bfd0>],
 1: [<pymo.data.MocapData at 0x11c275c5010>],
 2: [<pymo.data.MocapData at 0x11c268c5790>],
 3: [<pymo.data.MocapData at 0x11c28167bd0>],
 4: [<pymo.data.MocapData at 0x11c29bbe350>],
 5: [<pymo.data.MocapData at 0x11c29ac5a10>],
 6: [<pymo.data.MocapData at 0x11c27640510>],
 7: [<pymo.data.MocapData at 0x11c28a42d50>],
 8: [<pymo.data.MocapData at 0x11c27ebf710>],
 9: [<pymo.data.MocapData at 0x11c27aeed10>],
 10: [<pymo.data.MocapData at 0x11c27f3f6d0>],
 11: [<pymo.data.MocapData at 0x11c27998550>],
 12: [<pymo.data.MocapData at 0x11c297426d0>],
 13: [<pymo.data.MocapData at 0x11c28192990>],
 14: [<pymo.data.MocapData at 0x11c21785350>]}

In [10]:
# Definir ind
ind = []
ind = np.array([i for i in range(len(pos_cent))])
print(ind)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [11]:
dist = SHECCO()
dist.plot_mocap(Ns='01', Nv=0, ind=ind)

AttributeError: 'numpy.ndarray' object has no attribute 'parse'